In [2]:
# Importing necessary libraries
import pandas as pd
import numpy as np

In [3]:
# Load CSV
steam_games = pd.read_csv("../data/games.csv")

## Basic EDA

In [4]:
steam_games.head(5)

,AppID,Name,Release date,Estimated owners,Peak CCU,Required age,Price,DiscountDLC count,About the game,Supported languages,...,Average playtime two weeks,Median playtime forever,Median playtime two weeks,Developers,Publishers,Categories,Genres,Tags,Screenshots,Movies
20200,Galactic Bowling,"Oct 21, 2008",0 - 20000,0,0,19.99,0,0,Galactic Bowling is an exaggerated and stylize...,['English'],...,0,0,0,Perpetual FX Creative,Perpetual FX Creative,"Single-player,Multi-player,Steam Achievements,...","Casual,Indie,Sports","Indie,Casual,Sports,Bowling",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
655370,Train Bandit,"Oct 12, 2017",0 - 20000,0,0,0.99,0,0,THE LAW!! Looks to be a showdown atop a train....,"['English', 'French', 'Italian', 'German', 'Sp...",...,0,0,0,Rusty Moyher,Wild Rooster,"Single-player,Steam Achievements,Full controll...","Action,Indie","Indie,Action,Pixel Graphics,2D,Retro,Arcade,Sc...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
1732930,Jolt Project,"Nov 17, 2021",0 - 20000,0,0,4.99,0,0,Jolt Project: The army now has a new robotics ...,"['English', 'Portuguese - Brazil']",...,0,0,0,Campião Games,Campião Games,Single-player,"Action,Adventure,Indie,Strategy",NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
1355720,Henosis™,"Jul 23, 2020",0 - 20000,0,0,5.99,0,0,HENOSIS™ is a mysterious 2D Platform Puzzler w...,"['English', 'French', 'Italian', 'German', 'Sp...",...,0,0,0,Odd Critter Games,Odd Critter Games,"Single-player,Full controller support","Adventure,Casual,Indie","2D Platformer,Atmospheric,Surreal,Mystery,Puzz...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
1139950,Two Weeks in Painland,"Feb 3, 2020",0 - 20000,0,0,0.00,0,0,ABOUT THE GAME Play as a hacker who has arrang...,"['English', 'Spanish - Spain']",...,0,0,0,Unusual Games,Unusual Games,"Single-player,Steam Achievements","Adventure,Indie","Indie,Adventure,Nudity,Violent,Sexual Content,...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...


## Correcting Feature names
It seems like the feature names have been named incorrectly

In [ ]:
steam_games.columns = ['Name', 'Release date', 'Estimated owners', 'Peak CCU',
       'Required age', 'Price', 'Discount', 'DLC count', 'About the game',
       'Supported languages', 'Full audio languages', 'Reviews',
       'Header image', 'Website', 'Support url', 'Support email', 'Windows',
       'Mac', 'Linux', 'Metacritic score', 'Metacritic url', 'User score',
       'Positive', 'Negative', 'Score rank', 'Achievements', 'Recommendations',
       'Notes', 'Average playtime forever', 'Average playtime two weeks',
       'Median playtime forever', 'Median playtime two weeks', 'Developers',
       'Publishers', 'Categories', 'Genres', 'Tags', 'Screenshots', 'Movies']

In [ ]:
steam_games.head(3)

### Dropping unneeded columns

In [ ]:
steam_games.drop(["Screenshots",  "Movies", "Tags", "Website", "Support email", "Header image", "About the game" , "Reviews" , "Metacritic url", "Support url", "Score rank", "Notes", "Reviews", "About the game"], axis=1, inplace=True)

### Dropping all rows with Nulls

In [ ]:
null_cols = steam_games.isnull().sum()
null_cols = null_cols[null_cols > 0]
null_cols

In [ ]:
steam_games.dropna(inplace=True)

### Date Conversion

In this dataset, the release dates are not in the correct format.

In [ ]:
steam_games["Release date"].dtype

In [ ]:
pd.to_datetime(steam_games["Release date"], errors='coerce')

## Working with cleaned dataset

In [ ]:
steam_games.head(5)

In [ ]:
# Top 10 Peak CCUs
steam_games[['Name', 'Peak CCU']].sort_values(by='Peak CCU', ascending=False).head(10)

In [ ]:
# Most common game prices
steam_games['Price'].value_counts().head(5)

In [ ]:
steam_games["User score"].dtype

In [ ]:
steam_games['Estimated owners_str'] = steam_games['Estimated owners'].copy()
owners_split = steam_games["Estimated owners"].astype(str).str.split(' - ', expand=True)
owners_split_numeric = owners_split.apply(pd.to_numeric, errors='coerce')
steam_games["Estimated owners avg"] = owners_split_numeric.mean(axis=1)

In [ ]:
steam_games.loc[steam_games["Price"] > 0, "Value"] = steam_games["Average playtime forever"] / steam_games["Price"]


valid_mask = (
    (steam_games["Price"] > 0) &
    np.isfinite(steam_games["Value"]) &
    steam_games["Metacritic score"].notna() &
    (steam_games["Metacritic score"] > 0)
)

steam_games.loc[valid_mask, "Weighted Value"] = steam_games.loc[valid_mask, "Value"] * steam_games.loc[valid_mask, "Metacritic score"]

filtered = steam_games[
    (steam_games["Price"] > 10) &
    (steam_games["Recommendations"] > 10000)
]
filtered[["Name", "Weighted Value", "Metacritic score"]].sort_values(by="Weighted Value", ascending=False).head(10)

In [ ]:
steam_games["Release date"] = pd.to_datetime(steam_games["Release date"], errors='coerce')
steam_games.dropna(subset=["Release date"], inplace=True)

In [ ]:
steam_games["Release Year"] = steam_games["Release date"].dt.year

In [ ]:
price_conditions = [
    (steam_games['Price'] == 0),
    (steam_games['Price'] > 0) & (steam_games['Price'] <= 4.99),
    (steam_games['Price'] > 4.99) & (steam_games['Price'] <= 9.99),
    (steam_games['Price'] > 9.99) & (steam_games['Price'] <= 19.99),
    (steam_games['Price'] > 19.99) & (steam_games['Price'] <= 39.99),
    (steam_games['Price'] > 39.99) & (steam_games['Price'] <= 59.99),
    (steam_games['Price'] > 59.99)
]

choices = [
    'Free',
    '$0.01 - $4.99',
    '$5.00 - $9.99',
    '$10.00 - $19.99',
    '$20.00 - $39.99',
    '$40.00 - $59.99',
    '$60.00+'
]

steam_games['Price Category'] = np.select(price_conditions, choices, default="Unknown")

steam_games['Price Category'].value_counts()

In [ ]:
steam_games["Achievements"].sort_values(ascending=False).head(10)

In [ ]:
steam_games.columns

In [ ]:
steam_games['Steam_URL'] = 'https://store.steampowered.com/app/' + steam_games.index.astype(str) + '/'
steam_games['Steam_URL'].head(10)

In [ ]:
steam_games["Reviews"] = steam_games["Positive"] + steam_games["Negative"]
steam_games[["Name", "Reviews", "Price", "Discount"]].sort_values(by="Reviews", ascending=False).head(10)

In [ ]:
steam_games[steam_games["Name"] == "Grand Theft Auto V"]